# Basic text analytics: counting words

In this notebook, you will learn:
1. how to prepare text data for analysis
2. how to derive word frequency information for a corpus (a collection of texts)
3. how to produce a visualisation comparing the frequency of occurrence of words in two corpora

**Prerequisites**:
1. basic familiarity with R
3. The following R packages installed: **tm, NLP, stringi, magrittr**

### What's in a word?

The title above includes the phrase *counting words* - but what does *word* mean here? How would you define *word*?

To get a sense of the problem, think about the following question:

*How many words are there in the sentence '**The cat sat on the mat**'?*

There are two reasonable answers to this question:
- Answer 1: There are six words – six groups of characters separated in accordance with orthographic convention
- Answer 2: There are five words – five unique groups of characters and one of those occurs twice
Both answers are correct – but we are counting different things. The standard terms for these things are **tokens** and **types**.

Types are the unique entities on our sample. If we count types, we are not interested in how many times each type occurs; after we have included a type in our count, we ignore repetitions. If we count tokens, we do care how often each type occurs; we count each repetition of each type

We can also think about another question:

*Are 'dog' and dogs' the same word?*

They are distinct types – they are not identical aAnd they have to be distinct tokens (if they occur in the same text).
But we still have an intuition that there is some abstract entity which includes both of them - for example, we would not expect a (monolingual) dictionary to have entries for both. The abstract entity goes by various names such as **lemma** and **listeme**. (We are not going to look at such entities today.)

### Getting set up
We will be using four R packages in this exercise: **tm** which provides tools for text mining, **NLP** which tm requires, **stringi** which provides string processing tools, and **magrittr** which allows us to use pipe notation (more about this later). We do not want R to treat bits of text as factors, so we set the option which prevents that. We will be working with the data files which you created in the first part of the workshop and we assume that these are still locally available.

In [ ]:
# Set up - Colab
#install.packages('NLP')
#install.packages('tm')
#install.packages('stringi')
#install.packages('magrittr')
#library("NLP")
#library('tm')
#library('stringi')
#library('magrittr')

# Set up - SWAN
# create a local folder for the local files to live in
# [Creates a folder call "R" in your CloudStor home]
dir.create('/scratch/R', showWarnings = FALSE)

# add the local folder to R's
.libPaths(new='/scratch/R')

# install packages locally
install.packages("NLP", lib="/scratch/R")
install.packages("tm", lib="/scratch/R")
install.packages("stringi", lib="/scratch/R")
install.packages("magrittr", lib="/scratch/R")
# load package
library(NLP, lib="/scratch/R")
library(tm, lib="/scratch/R")
library(stringi, lib="/scratch/R")
library(magrittr, lib="/scratch/R") 
# set options
options(stringsAsFactors = F)

### Preparing the text
First, we have to make our text available for analysis. There are several preprocessing operations which are more or less standard in text analysis, but first we need to think about how these operations relate to twitter data.

The details of how text is tokenised vary from one tool to another. This means that in order to have *dog* and *dog,* treated as tokens of the same type, removing punctuation from text is sensible. This is normally done using a predefined list of punctuation characters such as this one:

! " # $ % & ' * + , . ) ( : ; < = > ? @ \ / ] [ ^ ` { | } ~

But this might be problematic for Twitter data! If we delete all punctuation from tweets, we will not be able to keep track of two important elements:
- Hashtags e.g. #auspol
- Handles e.g. @SimonMusgrave1

One solution would be to change the relevant punctuation to text, but this is a bit clunky. Another possibility is to not delete those two punctuation marks. Text handling packages (like **tm** and **OpenNLP**) come with predefined punctuation lists and (in theory) we can edit those lists. But in practice it can be hard to track them down and it is easier to write a line of code to do a customised deletion.

The next code cell does some basic cleaning operations on our data, such as getting rid of urls and new line characers (and a solution for the punctuation problem). We end by inspecting the data to make sure that we have the results we expect.

In [ ]:
# read data as csv file, read text as vector, create corpus
# assumes data file is in the working directory
echidna <- read.csv("echidna_minimal.csv", header = TRUE, stringsAsFactors = FALSE, encoding = "UTF-8")
# remove urls
echidna$stripped_text <- gsub("http.*","",  echidna$text)
echidna$stripped_text <- gsub("https.*","", echidna$stripped_text)
# remove new lines
echidna$stripped_text <- gsub("\\n"," ", echidna$stripped_text, fixed = TRUE)
# punctuation 
echidna$stripped_text <- gsub("['.,!?\"*+%&;:<=>/)(^~`|)(}{]", "", echidna$stripped_text)

# inspect data
str(echidna)

We also need to do something about emojis - they are part of the communicative resources of twitter and we don't want to just ignore them. The way we will do this is by replacing the emojis with their prose description. Notice that we have read in the data as UTF-8, i.e. Unicode encoding. We can search for the Unicode representations of emojis (which look like this: \U+1F928) and then recover the description from the look-up table we download. Those descriptions have spaces in them and we delete those - our counting procedures look at single tokens, so *smiling face* will not be recorded as a unit but *smilingface* will. (Note that even if we decided we were not interested in emojis - and that is a reasonable decision in some cases - we would still need to be able to identify them in order to delete them.)

The various search-and-replace operations in the previous and following code blocks use **regular expressions**. Regular expressions are a powerful way of specifying patterns of characters to be matched; if you would like to learn more about them, [this](http://solaris-8.tripod.com/regexp.pdf) is a good place to start.

This code block also uses *piping*, mentioned above. The symbol **%>%** shows this - it can be read more or less as *and then*. So the first snippet which starts with *emoji* can be read as:
Take the object **emoji**, extract all the elements in that object which match the specified pattern, *and then* take the output of that operation and replace the specified pattern with nothing *and then* take the output of that operation and trim any space from the strings *and then* assign the output to the object **emoji.chars**.

In [ ]:
# encoded emojis to prose descriptions
# read emoji info and get rid of documentation lines
readLines("https://unicode.org/Public/emoji/5.0/emoji-test.txt",
          encoding="UTF-8") %>%
  stri_subset_regex(pattern = "^[^#]") %>%
  stri_subset_regex(pattern = ".+") -> emoji

# get the emoji characters and clean them up
emoji %>%
  stri_extract_all_regex(pattern = "# *.{1,2} *") %>%
  stri_replace_all_fixed(pattern = c("*", "#"),
                         replacement = "",
                         vectorize_all=FALSE) %>%
  stri_trim_both() -> emoji.chars

# get the emoji character descriptions
emoji %>%
  stri_extract_all_regex(pattern = "#.*$") %>%
  stri_replace_all_regex(pattern = "# *.{1,2} *",
                         replacement = "") %>%
  stri_trim_both() -> emoji.descriptions
# collapse the descriptions
emoji.descriptions <- gsub(" ", '', emoji.descriptions, fixed = TRUE)

# Replace Unicode characters with prose descriptions
echidna$stripped_text <- stri_replace_all_regex(echidna$stripped_text, pattern = emoji.chars, replacement = paste(emoji.descriptions," "), vectorize_all=FALSE)

# inspect data
head(echidna$stripped_text)

### More preparation
Now we can do the kind of preparation which we might do for any text data. Creating a Corpus object via **tm** is a good way to do this; it means we can  use **tm** tools to do some pre-processing and we will use other **tm** tools to do the counting later.

Initially, we will apply four kinds of cleaning to our data:
1. Making all characters lower case; otherwise *dog* and *Dog* will be treated as distinct types.
2. Removing numbers; probably not important here, but e.g. page numbers can cause artefacts to appear in analyses
3. Removing whitespace other than single spaces between tokens.

(You will see some warning messages as this code runs - they are not important.)

In [ ]:
# make corpus
echidna_corpus <- Corpus(VectorSource(echidna$stripped_text))

# all lower case, remove whitespace, numbers
echidna_corpus <- echidna_corpus %>% tm_map(content_transformer(tolower)) %>% tm_map(removePunctuation) %>% tm_map(removeNumbers) %>% tm_map(stripWhitespace)


### Doing some counting
Now we can get the most basic numbers for our corpus: how many tokens and how many types do we have? We will do this by creating a matrix where each row represents a document and each column represents a type. The number of columns is the number of types, and we can get the number of tokens either as the sum of the sums of each row or as the sum of the sums of each column.

In [ ]:
# create document-term matrix
dtm <- DocumentTermMatrix(echidna_corpus)
# get word frequency list
# token count 
tokens1 <- sum(rowSums(as.matrix(dtm)))
print(paste0("Tokens: ", tokens1))
# collapse matrix by summing over columns
freq <- colSums(as.matrix(dtm))
# length should be total number of terms - type count
print(paste0('Types: ',length(freq)))

### Looking at frequency
Now we can look at what are the most frequently occurring types in our corpus. We take the column sums, the number of tokens for each type, and put them in a new matrix. Then we sort the matrix in descending order and view the 20 most frequent types.

In [ ]:
# create sort order (descending)
ord <- order(freq,decreasing=TRUE)
# List top 20 terms in decreasing order of freq
freq[ord][0:20]


### Function words and content words
These results are what we would expect to see for most samples of English text: function words like *the* and *and* are very common. The 20 most frequent words in the British National Corpus are *the, of, and, a, in, to, it, is, was, to, i, for, you, he, be, with, on, that, by, at*, all function words. 

Are there differences between the BNC list and the most frequent words in our data? Can we make any sense of these differences?

In text analysis, it is very common to use a **stop list**, that is a list of words which will removed from the corpus before any analysis is performed. We will apply a stop list to our corpus and see what difference this makes to the list of most frequent words. (You can ignore the warning message again)

In [ ]:
# remove stopwords
docs1_stop <- tm_map(echidna_corpus, removeWords, stopwords("english"))
dtm <- DocumentTermMatrix(docs1_stop)
freq <- colSums(as.matrix(dtm))
ord <- order(freq,decreasing=TRUE)
freq[ord][0:20]

### Comparing frequency
We want to compare frequencies across two corpora, but we cannot do this by just comparing the counts directly because the corpora may differ in size. We expect that there will be more tokens of a given type in a larger corpus than in a smaller corpus just because of the difference in size, but we are interested in the relative frequency of types in the two corpora. To make this comparison possible, we need to **normalise** our frequency counts to some fixed number of tokens. Here, we will normalise frequency per 100k tokens. We also save the data for the 20 most common types as a data frame - we will use this when we make a visual comparison of the two text collections.

In [ ]:
# make freq a data frame, add normalised freq
types <- names(freq)
freq_df <- data.frame(types, freq)
ord_freq_C1 <- freq_df[order(freq, decreasing = TRUE),]
ord_freq_C1['norm_freq'] <- ord_freq_C1$freq/(tokens1/100000)
ord_freq_C1[0:20,]
# store top 20 words for future use
top20_echidna <- ord_freq_C1[0:20,]

### Getting data from the second corpus

Now, we will repeat our procedures for the second corpus - we won't bother with showing the frequent types without the stop list. 

In [ ]:
# corpus2
# read data as csv file, read text as vector, create corpus
platypus <- read.csv("platypus_minimal.csv", header = TRUE, stringsAsFactors = FALSE, encoding = "UTF-8")
# remove urls
platypus$stripped_text <- gsub("http.*","",  platypus$text)
platypus$stripped_text <- gsub("https.*","", platypus$stripped_text)
# remove new lines
platypus$stripped_text <- gsub("\\n"," ", platypus$stripped_text, fixed = TRUE)
# change # and @ to text 
platypus$stripped_text <- gsub("#","HASH", platypus$stripped_text)
platypus$stripped_text <- gsub("@","AT", platypus$stripped_text)

platypus$stripped_text <- stri_replace_all_regex(platypus$stripped_text, pattern = emoji.chars, replacement = paste(emoji.descriptions," "), vectorize_all=FALSE)

# make corpus
platypus_corpus <- Corpus(VectorSource(platypus$stripped_text))

# all lower case,remove punctuation, numbers
platypus_corpus <- platypus_corpus %>% tm_map(content_transformer(tolower)) %>% tm_map(removePunctuation) %>% tm_map(removeNumbers) %>% tm_map(stripWhitespace)

# create document-term matrix
dtm <- DocumentTermMatrix(platypus_corpus)
#get word frequency list
# token count 
tokens2 <- sum(rowSums(as.matrix(dtm)))
print(paste0("Tokens: ", tokens2))
# remove stopwords
docs2_stop <- tm_map(platypus_corpus, removeWords, stopwords("english"))
# create document-term matrix
dtm <- DocumentTermMatrix(docs2_stop)
# collapse matrix by summing over columns
freq <- colSums(as.matrix(dtm))
# length should be total number of terms - type count
print(paste0('Types: ',length(freq)))
# create sort order (descending)
ord <- order(freq,decreasing=TRUE)
# make freq a data frame, add normalised freq
types <- names(freq)
freq_df <- data.frame(types, freq)
ord_freq_C2 <- freq_df[order(freq, decreasing = TRUE),]
ord_freq_C2['norm_freq'] <- ord_freq_C2$freq/(tokens2/100000)
ord_freq_C2[0:20,]
# inspect data
top20_C2 <- ord_freq_C2[0:20,]

### Making a visualisation
The lists of 20 most frequent types overlap a little, but they are certainly not identical. We want to include all the words from both lists, so we combine them and then use the **unique()** function to get rid of duplicates. A side-by-side bar chart is a good way to visualise this data; the R base function **barplot()** will do this, but it needs the data as rows of a table.

In [ ]:
# combine the two lists of common types
types_combined <- top20_echidna['types']
types_combined <- rbind(types_combined, top20_C2['types'])
types_combined <- unique(types_combined)
# make a matrix with the frequency data in two rows
rows <- nrow(types_combined)
bar_data <- matrix(1:rows*2, nrow = 2, ncol = rows)
# get the normalised frequency values
for (i in 1:rows) {
    word <- types_combined[i,1]
    freq_1 <- subset(ord_freq_C1, types == word)[3]
    bar_data[1,i] <- freq_1[1,1]
    freq_2 <- subset(ord_freq_C2, types == word)[3]
    bar_data[2,i] <- freq_2[1,1]
    }

rows <- c('C1', 'C2')
columns <- types_combined[,1]
rownames(bar_data) <- rows
colnames(bar_data) <- columns
# inspect data
bar_data

We need to display the types vertically or we won't be able to see them all. Some of them are quite long words, so we reset the lower margin to make sure all of the characters will be visible. We also use the **las** parameter to make the text on each axis perpendicular to the axis. And then all that is left to do is to choose some colours that you like!

In [ ]:
# set bottom margin to fit long words
options(jupyter.plot_scale=1)
par(mar=c(8,4,4,4))
barplot(bar_data, beside = TRUE, las = 2, cex.names = 0.6,
main = "Comparative normalised frequencies (per100k)", col = c("cyan","grey"))
legend("topright",
c("Echidna","Platypus"),
fill = c("cyan","grey"))
axis(2, las=1)

### Outro

Extract session information

In [ ]:
sessionInfo()